In [1]:
import cv2
import csv
import os
from tqdm import tqdm_notebook as tqdm
import numpy as np
from pandas import DataFrame

## Name of directory
### Enter the name of the directory from which you want to parse the tsv files
### And the name of the new directory where the files will be saved

In [2]:
directory = 'Annotations/'
new_directory = "Annotations_with_circle"

In [3]:
def triangle(coordinates):
    points = [[int(coordinates[0][0]),int(coordinates[0][1])], \
              [int(coordinates[1][0]),int(coordinates[0][1])], \
              [(int(coordinates[0][0])+int(coordinates[1][0]))//2, int(coordinates[1][1])]]
    return points
def romb(coordinates):
    points = [[(int(coordinates[0][0])+int(coordinates[1][0]))//2, int(coordinates[0][1])], \
              [int(coordinates[1][0]), (int(coordinates[0][1])+int(coordinates[1][1]))//2], \
              [(int(coordinates[0][0])+int(coordinates[1][0]))//2, int(coordinates[1][1])], \
              [int(coordinates[0][0]), (int(coordinates[0][1])+int(coordinates[1][1]))//2]]
    return points
def circle(coordinates):
    points = [[(int(coordinates[0][0])+int(coordinates[1][0]))//2, \
               (int(coordinates[0][1])+int(coordinates[1][1]))//2],\
               (int(coordinates[1][0])-int(coordinates[0][0]))//2]
    return points
def rectangle(coordinates):
    ltp = (int(coordinates[0][0]), int(coordinates[0][1]))
    rbp = (int(coordinates[1][0]), int(coordinates[1][1]))
    return [ltp, rbp]
def ellipse(coordinates):
    points = [[(int(coordinates[0][0])+int(coordinates[1][0]))//2, \
               (int(coordinates[0][1])+int(coordinates[1][1]))//2],\
              [(int(coordinates[1][0])-int(coordinates[0][0]))//2,\
               (int(coordinates[1][1])-int(coordinates[0][1]))//2]]
    return points

In [4]:
def box_to_mask(coordinates):
    label = (coordinates[2])
    if label == '2.1':
        return romb(coordinates)
    elif label == '2.4':
        return triangle(coordinates)
    elif ((label=='3.1')|(label=='3.24')|(label=='3.27')|(label=='4.1')|(label=='4.2.1')|(label=='4.2.2')|(label=='4.2.3')|(label=='4.1.1')|(label=='4.1.2')|(label=='4.1.3')|(label=='4.1.4')|(label=='4.1.5')|(label=='4.1.6')):
        return circle(coordinates) #ellipse(coordinates)
    else:
        return rectangle(coordinates)      
def put_mask(mask, label):
    if ((label == '2.1') | (label == '2.4')):
        cv2.polylines(output,mask,True,(0,255,255))
    elif ((label=='3.1')|(label=='3.24')|(label=='3.27')|(label=='4.1')|(label=='4.2.1')|(label=='4.2.2')|(label=='4.2.3')|(label=='4.1.1')|(label=='4.1.2')|(label=='4.1.3')|(label=='4.1.4')|(label=='4.1.5')|(label=='4.1.6')):
        lol = tuple(mask[0])
        cv2.circle(output,tuple(mask[0]), mask[1]+1, (0,0,255), -1)
    else :
        cv2.rectangle(output,mask[0],mask[1] , (0, 0, 255), 1)
def type_of_sign(label):
    if label == '2.1':
        return "romb"
    elif label == '2.4':
        return "triangle"
    elif ((label=='3.1')|(label=='3.24')|(label=='3.27')|(label=='4.1')|(label=='4.2.1')|(label=='4.2.2')|(label=='4.2.3')|(label=='4.1.1')|(label=='4.1.2')|(label=='4.1.3')|(label=='4.1.4')|(label=='4.1.5')|(label=='4.1.6')):
        return "circle"    #"ellipse"
    else:
        return "rectangle"     

In [8]:
for filename in tqdm(os.listdir(directory)):
    rect = list()
    with open(directory+"/"+filename) as tsvfile:
        tsvreader = csv.reader(tsvfile, delimiter="\t")
        for line in tsvreader:
            rect.append([line[1:3],line[3:5], line[0]])
    del rect[0]
    df = DataFrame.from_csv(directory+"/"+filename, sep="\t")
    df['form'] = 'string'
    df['coordinates'] = np.array
    for i, coordinates in enumerate(rect):
        mask = box_to_mask(coordinates)
        df.iat[i, 7]=type_of_sign(coordinates[2])
        df.iat[i, 8]=mask
        df.to_csv(new_directory+"/"+filename)
    del rect

/home/andreizoltan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


In [9]:
df

,xtl,ytl,xbr,ybr,temporary,occluded,data,form,coordinates
class,,,,,,,,,
6.10.1,1760,480,2356,791,False,False,NaN,rectangle,"[(1760, 480), (2356, 791)]"
8.13,1276,847,1327,899,False,False,NaN,rectangle,"[(1276, 847), (1327, 899)]"
3.24,1243,957,1248,961,False,True,NaN,circle,"[[1245, 959], 2]"
5.20,1243,942,1253,954,False,True,NaN,rectangle,"[(1243, 942), (1253, 954)]"
5.19.2,1071,954,1081,959,False,True,NaN,rectangle,"[(1071, 954), (1081, 959)]"
1.34.1,1071,960,1155,986,False,False,NaN,rectangle,"[(1071, 960), (1155, 986)]"
5.19.1,226,954,240,969,False,False,NaN,rectangle,"[(226, 954), (240, 969)]"
2.1,225,931,243,952,False,False,NaN,romb,"[[234, 931], [243, 941], [234, 952], [225, 941]]"
4.3,1275,901,1325,952,False,False,NaN,rectangle,"[(1275, 901), (1325, 952)]"
